<a href="https://colab.research.google.com/github/rkh97/ML-foundations/blob/master/ch6_kyunghee_%E1%84%92%E1%85%A1%E1%86%B8%E1%84%89%E1%85%A5%E1%86%BC%E1%84%80%E1%85%A9%E1%86%B8%E1%84%89%E1%85%B5%E1%86%AB%E1%84%80%E1%85%A7%E1%86%BC%E1%84%86%E1%85%A1%E1%86%BC_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. 합성곱 신경망

### 6.1 합성곱 연산과 풀링 연산


### 6.2 AlexNet

In [ ]:
# 라이브러리
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# CIFAR10 데이터 세트 불러오기
# transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform = transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size = 32, shuffle=True)

# testset = torchvision.datasets.CIFAR10(root='./data',train=False, download=True, transform= transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)

In [ ]:
# GPU 연산 체크하기
device = torch.device("cuda : o" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

In [ ]:
# AlexNet 구축하기
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential( # 순차적으로 행해지는 연산을 한 번에 묶는다. 작성 순서대로 연산이 수행됨
            nn.Conv2d(3,64,3), nn.ReLU(), # 합성곱 연산과 풀링 연산이 행해지는 feature extraction 부분
            nn.MaxPool2d(2,2),
            nn.Conv2d(64,192,3, padding=1), nn.ReLU(), # nn.Conv2d(입력채널수, 출력채널수, 필터크기=이미지니까 3)
            nn.MaxPool2d(2,2), # nn.MaxPool2d(필터크기, 보폭)
            nn.Conv2d(192,384,3, padding=1), nn.ReLU(), # padding=1 : 해당 층의 입력 피쳐맵의 외각을 0으로 간 겹 둘러싼다는 의미
            nn.Conv2d(384,256,3, padding=1), nn.ReLU(),
            nn.Conv2d(256,256,1), nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.classifier = nn.Sequential( # Fully-connected layer
            nn.Dropout(0.5),
            nn.Linear(256*3*3,1024), nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024,512), nn.ReLU(),
            nn.Linear(512,10), nn.ReLU() # 10개의 클래스를 가진 데이터이므로 노드 수는 10
        )

    def forward(self,x):
        x = self.features(x)
        x = x.view(-1,256*3*3)
        x = self.classifier(x)

        return x

In [ ]:
# 손실 함수 및 최적화 방법 정의하기
criterion = nn.CrossEntropyLoss() # 다중 분류 문제에서는 주로 크로스엔트로피 이용
alexnet = AlexNet().to(device) # GPU 연산을 위해 모델을 불러옴
optimizer = optim.Adam(alexnet.parameters(),lr=1e-3)

In [ ]:
# AlexNet 모델 학습
loss_= [] # 그래프를 그리기 위한 loss 저장용 리스트
n = len(trainloader) # 배치개수
for epoch in range(50):
    running_loss = 0.0
    for data in trainloader:
        inputs, labels = data[0].to(device), data[1].to(device) # 배치 데이터
        optimizer.zero_grad()
        outputs = alexnet(inputs) # 예측값 산출
        loss = criterion(outputs, labels) # 손실 함수 계산
        loss.backward() # 손실함수 기준으로 역전파 선언
        optimizer.step() # 가중치 최적화
        running_loss += loss.item()

    loss_.append(running_loss/n)
    print('[%d] loss : %.3f' % (epoch + 1, running_loss/len(trainloader)))

In [ ]:
plt.plot(loss_)
plt.title("training loss")
plt.xlabel("epoch")
plt.show()

In [ ]:
path = './model/cifar_alexnet.pth'
torch.save(alexnet.state_dic(),path)

alexnet = AlexNet.to(device)
alexnet.load_state_dict(torch.load(path))

### 6.3 ResNet
- skip connection이란 여러 레이어를 건너 뛰어 이전 정보를 더하는 것을 의미
- 이 하나의 과정을 묶어 만든 것이 residual block
- ResNet은 Residual block 여러 개를 붙여놓은 모델
- 모델 명에 붙은 숫자는 층의 개수를 의미

In [ ]:
# residual block 구축하기
class ResidualBlock(nn.Module):

    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.stride = stride
        self.in_channels - in_channels
        self.out_channels = out_channels
        self.conv_block = nn.Sequential(
            nn.Conv2d(self,in_channels,self.out_channels, kernel_size =3, sride=stride, padding=1, bias =False),
            nn.BatchNorm2d(self,out_channels), # 배치 정규화를 층사이에 적용해서 학습을 빠르게 한다. 배치 정규화는 각 배치의 평균과 분산을 이용해서 데이터를 정규화하는 방법
            nn.ReLU(),
            nn.Conv2d(self,in_channels,self.out_channels, kernel_size =3, sride=stride, padding=1, bias =False),
            nn.BatchNorm2d(self,out_channels))

        if self.stride != 1 or self.in_channels != self.out_channels:
            self.downsample = nn.Sequential(
                            nn.Conv2d(self.in_channels,self.out_channels,kernel_size=1,stride=stride, bias=False),
                            nn.BatchNorm2d(self.out_channels)
            )

    def forward(self,x):
        out = self.conv_block(x)
        if self.stride != 1 of self.in_channels != self.out_channels:
            x = self.downsample(x)
        out = F.relu(x+out)
        return out



In [ ]:
# ResNet 모델 구축하기
class ResNet(nn.Module):
    def __init__(self, num_blocks, num_classes =10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.base = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer1 = self._make_layer(64,num_blocks[0],stride=1)
        self.layer2 = self._make_layer(128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(512, num_blocks[3], stride=2)
        self.gap = nn.AvgPool2d(4) # 필터사이즈 4
        self.fc = nn.Linear(512,num_classes)

    def _make_layer(self, out_channels, num_blocks, stride):
        strides =[stride]+[1]*(num_blocks-1)
        layers=[]
        for stride in strides:
            block = ResidualBlock(self.in_channels, out_channels, stride)
            layers.append(block)
            self.in_channels = out_channels
        return nn.Sequential(*layers)


In [ ]:
def forward(self, x):
    out = self.base(x)
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.gap(out)
    out = out.view(out.size(0),-1)
    out = self.fc(out)
    return out

In [ ]:
def modeltype(model):
    if model == 'resnet18':
        return ResNet([2,2,2,2])
    elif model == 'resnet34':
        return ResNet([3,4,6,3])

resnet = modeltype('resnet18').to(device)
print(resnet)
path = './models/cifar_resnet.pth'